### ML Model for Stock Return Prediction
Using Richard's ceo_gender_file
Incorporates certain hand-crafted keywords as additional variables

In [1]:
import pandas as pd
import numpy as np
from numpy import loadtxt
import keras
from keras.models import Sequential
from keras.layers import Dense
import os
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
import re
from sklearn.feature_extraction.text import CountVectorizer
import gensim
import nltk
import string
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
def cleantxt(txt):
    """
    Cleans the string passed. Cleaning Includes-
    1. remove special characters/symbols
    2. convert text to lower-case
    3. retain only alphabets
    4. remove words less than 3 characters
    5. remove stop-words
    """  
    # collecting english stop words from nltk-library
    stpw = stopwords.words('english')
    
    # Adding custom stop-words
    stpw.extend(['www','http','utc'])
    stpw = set(stpw)
    
    # using regex to clean the text
    txt = re.sub(r"\n", " ", txt)
    txt = re.sub("[\<\[].*?[\>\]]", " ", txt)
    txt = txt.lower()
    txt = re.sub(r"[^a-z ]", " ", txt)
    txt = re.sub(r"\b\w{1,3}\b", " ",txt)
    txt = " ".join([x for x in txt.split() if x not in stpw])
    return txt


In [8]:
# Read in additional keywords
esg_keywords = pd.read_excel("Keywords_ESG.xlsx", index_col=None, header=None)
esg_keywords = set(esg_keywords[0].unique())
esg_keywords

{'biodiversity',
 'carbon',
 'carbon neutral',
 'clean-up',
 'climate change',
 'code of ethic',
 'corporate culture',
 'corporate governance',
 'corruption',
 'discharge',
 'diversity',
 'divestment',
 'emission',
 'employee relation',
 'enevery-efficiant',
 'environmental',
 'equality',
 'esg',
 'ethical',
 'ethnics',
 'fuel efficiency',
 'gender',
 'gender equality',
 'governance risk',
 'green energy',
 'greenhouse',
 'hazardous',
 'health',
 'human right',
 'labor relation',
 'labor standard',
 'low carbon',
 'natural resource',
 'net-zero emission',
 'pollution',
 'race',
 'remediation',
 'renewable',
 'renewable energy',
 'responsible',
 'safety',
 'scandal',
 'social',
 'societal ',
 'societal impact',
 'sustainability',
 'sustainable',
 'tocis',
 'transparency',
 'waste',
 'working condition',
 'zero carbon'}

In [52]:
# Read in keywords with extremity rating
keywords_with_rating = pd.read_excel("keywords_conference.xlsx", index_col=None, header=0)
keywords_with_rating
rate_5 = keywords_with_rating[5].dropna().unique()
rate_4 = keywords_with_rating[4].dropna().unique()
rate_3 = keywords_with_rating[3].dropna().unique()
rate_2 = keywords_with_rating[2].dropna().unique()
rate_1 = keywords_with_rating[1].dropna().unique()
rate_minus1 = keywords_with_rating[-1].dropna().unique()
rate_minus2 = keywords_with_rating[-2].dropna().unique()
rate_minus3 = keywords_with_rating[-3].dropna().unique()
rate_minus4 = keywords_with_rating[-4].dropna().unique()
rate_minus5 = keywords_with_rating[-5].dropna().unique()
rate_1
#keyword_rate_dict = {rate_5: 5, rate_4: 4, rate_3: 3, rate_2: 2, rate_1: 1, rate_minus1: -1,
                     #rate_minus2: -2, rate_minus3: -3, rate_minus4: -4, rate_minus5: -5}

array(['generate', 'competitive', 'produce', 'steady', 'encourage',
       'transparency', 'sufficient'], dtype=object)

In [99]:
# create an array with (CIK: strategy text)
corpus_dict = dict()
corpus = [] # list of strategy text
ncols = 0

with open("ceo_gender_training.csv") as f:
    ncols = len(f.readline().split(','))
ceo_data = pd.read_csv("ceo_gender_training.csv")
ceo_data.rename(columns={ceo_data.columns[0]: "pid" }, inplace = True)
ceo_data['has_file'] = 0
company_doc = []

for i in range(-5, 6, 1):
    if i == 0: continue
    ceo_data['keyword_count_'+ str(i)] = 0
ceo_data['total_extreme'] = 0
ceo_data['total_moderate'] = 0
ceo_data['signed_extreme'] = 0
ceo_data['signed_moderate'] = 0

    
for index, row in ceo_data.iterrows():
    folder_path = row['folder_path']
    QTR = folder_path.split("\\")[0]
    CIK = folder_path.split("\\")[1]
    try:
        f1 = open(QTR + "/" + CIK + "/" + "business-section.txt", "r")
        business = f1.read()
        f2 = open(QTR + "/" + CIK + "/" + "risk-factors-section.txt", "r")
        risk = f2.read()
        strategy = business + risk
        ceo_data.at[index,'has_file'] = 1
        clean_strategy = cleantxt(strategy)
        corpus_dict[CIK] = clean_strategy
        corpus.append(clean_strategy)
        # report length and keywords
        report_length = len(strategy)
        ceo_data.at[index,'report_length'] = report_length
        esg_word_count = 0
        strategy_words = strategy.split()
        for word in esg_keywords:
            esg_word_count += strategy_words.count(word)
        esg_word_perc = esg_word_count / report_length
        ceo_data.at[index,'keyword_perc'] = esg_word_perc
        # Add keyword data with extremity rating
        for rating in range(-5, 6, 1):
            if rating == 0: continue
            word_count = 0
            rate_words = keywords_with_rating[rating].dropna().unique()
            for word in rate_words:
                word_count += strategy_words.count(word)
            col_name = 'keyword_count_'+ str(rating)
            ceo_data.at[index, col_name] = word_count
            
    except FileNotFoundError:
        ceo_data.at[index,'has_file'] = 0


ceo_data['has_file'].unique()
# consider only rows with the file data
ceo_data2 = ceo_data.loc[ceo_data['has_file'] == 1].reset_index()
#print(ceo_data2.shape)
ceo_data2

,index,pid,CIK,cusip,CompName,folder_name,folder_path,at,lt,che,...,keyword_count_2,keyword_count_3,keyword_count_4,keyword_count_5,total_extreme,total_moderate,signed_extreme,signed_moderate,report_length,keyword_perc
0,0,0,1000228.0,80640710,HENRY SCHEIN INC,0001000228-20-000018,QTR1\0001000228-20-000018,7151.101,3233.706,106.097,...,35,16,9,8,0,0,0,0,146854.0,0.001444
1,1,1,75252.0,69073210,OWENS & MINOR INC/VA/,0000075252-20-000021,QTR1\0000075252-20-000021,3643.084,3180.930,67.030,...,20,4,3,4,0,0,0,0,70848.0,0.000240
2,2,2,891024.0,70339510,"PATTERSON COMPANIES, INC.",0000891024-20-000008,QTR2\0000891024-20-000008,2715.350,1878.906,77.944,...,53,10,12,8,0,0,0,0,178435.0,0.000953
3,3,3,1000229.0,N2271710,CORE LABORATORIES N V,0001564590-20-004075,QTR1\0001564590-20-004075,774.673,592.533,11.092,...,21,4,8,3,0,0,0,0,65548.0,0.000809
4,5,5,1681459.0,G8711010,TechnipFMC plc,0001681459-20-000004,QTR1\0001681459-20-000004,23518.800,15789.600,5239.900,...,59,9,28,11,0,0,0,0,139875.0,0.000293
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1025,1227,1227,100493.0,90249410,"TYSON FOODS, INC.",0000100493-20-000132,QTR4\0000100493-20-000132,33097.000,18871.000,485.000,...,37,7,16,1,0,0,0,0,98413.0,0.000640
1026,1228,1228,48465.0,44045210,HORMEL FOODS CORP /DE/,0000048465-20-000043,QTR4\0000048465-20-000043,8109.004,2183.469,687.637,...,8,5,2,2,0,0,0,0,39129.0,0.000511
1027,1229,1229,47518.0,43147510,"Hill-Rom Holdings, Inc.",0000047518-20-000129,QTR4\0000047518-20-000129,4919.000,3345.700,633.800,...,21,9,6,2,0,0,0,0,75905.0,0.000962
1028,1230,1230,63296.0,57712810,MATTHEWS INTERNATIONAL CORP,0000063296-20-000098,QTR4\0000063296-20-000098,2190.603,1471.367,35.302,...,12,6,2,2,0,0,0,0,51732.0,0.000831


In [101]:
ceo_data2 = ceo_data2.dropna()
for index, row in ceo_data2.iterrows():
    rate_5 = ceo_data2.at[index, 'keyword_count_5']
    rate_4 = ceo_data2.at[index, 'keyword_count_4']
    rate_3 = ceo_data2.at[index, 'keyword_count_3']
    rate_2 = ceo_data2.at[index, 'keyword_count_2']
    rate_1 = ceo_data2.at[index, 'keyword_count_1']
    rate_minus1 = ceo_data2.at[index, 'keyword_count_-1']
    rate_minus2 = ceo_data2.at[index, 'keyword_count_-2']
    rate_minus3 = ceo_data2.at[index, 'keyword_count_-3']
    rate_minus4 = ceo_data2.at[index, 'keyword_count_-4']
    rate_minus5 = ceo_data2.at[index, 'keyword_count_-5']
    report_length = ceo_data2.at[index, 'report_length']
    ceo_data2.at[index, 'total_extreme'] = (rate_5 + rate_4) / report_length
    ceo_data2.at[index, 'total_moderate'] = (rate_1 + rate_2 + rate_3) / report_length
    ceo_data2.at[index, 'signed_extreme'] = (rate_4 + rate_5 - rate_minus4 - rate_minus5) / report_length
    ceo_data2.at[index, 'signed_moderate'] = (rate_1 + rate_2 + rate_3 + 
                                            rate_minus1 + rate_minus2 + rate_minus3) / report_length
    
ceo_data_small = ceo_data2[["ret_0220_0325", "at", "lt", "che", "ni", "sale", "execrank", "exchange_id", 
                            "gender_id", "state_id", "days_becameceo", "zip", "FF_48", "shrout_0220",
                            "report_length", "keyword_perc"]]
                            #"total_extreme", "total_moderate", "signed_extreme", "signed_moderate"]]

cleaned_data = ceo_data_small.dropna()
    
X = cleaned_data.iloc[:,1:]
y = cleaned_data.iloc[:,0:1]
# split trian and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
print(X_train.shape)
print(y_train.shape)

(756, 15)
(756, 1)


In [102]:
# define the keras model for deep learning
model = Sequential()
model.add(Dense(5, input_dim=15, activation='relu'))
model.add(Dense(4, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# compile the keras model
opt = keras.optimizers.Adam(learning_rate=0.01)
model.compile(loss='mean_squared_error', optimizer=opt, metrics=['mean_squared_error'])
# fit the keras model on the dataset
model.fit(X_train, y_train, epochs=150, batch_size=10)

# evaluate the keras model
# _, mse = model.evaluate(X_test, y_test)
# print('MSE: %.2f' % (mse*100))
results = model.evaluate(X_test, y_test, batch_size=128)
print("test loss, test mse:", results)


Epoch 1/150
76/76 [==============================] - 0s 643us/step - loss: 0.1980 - mean_squared_error: 0.1980
Epoch 2/150
76/76 [==============================] - 0s 612us/step - loss: 0.1530 - mean_squared_error: 0.1530
Epoch 3/150
76/76 [==============================] - 0s 603us/step - loss: 0.1399 - mean_squared_error: 0.1399
Epoch 4/150
76/76 [==============================] - 0s 667us/step - loss: 0.1338 - mean_squared_error: 0.1338
Epoch 5/150
76/76 [==============================] - 0s 651us/step - loss: 0.1325 - mean_squared_error: 0.1325
Epoch 6/150
76/76 [==============================] - 0s 644us/step - loss: 0.1364 - mean_squared_error: 0.1364
Epoch 7/150
76/76 [==============================] - 0s 600us/step - loss: 0.1378 - mean_squared_error: 0.1378
Epoch 8/150
76/76 [==============================] - 0s 613us/step - loss: 0.1388 - mean_squared_error: 0.1388
Epoch 9/150
76/76 [==============================] - 0s 628us/step - loss: 0.1412 - mean_squared_error: 0.1412
E

76/76 [==============================] - 0s 649us/step - loss: 0.1372 - mean_squared_error: 0.1372
Epoch 75/150
76/76 [==============================] - 0s 645us/step - loss: 0.1363 - mean_squared_error: 0.1363
Epoch 76/150
76/76 [==============================] - 0s 587us/step - loss: 0.1444 - mean_squared_error: 0.1444
Epoch 77/150
76/76 [==============================] - 0s 685us/step - loss: 0.1389 - mean_squared_error: 0.1389
Epoch 78/150
76/76 [==============================] - 0s 786us/step - loss: 0.1491 - mean_squared_error: 0.1491
Epoch 79/150
76/76 [==============================] - 0s 718us/step - loss: 0.1449 - mean_squared_error: 0.1449
Epoch 80/150
76/76 [==============================] - 0s 696us/step - loss: 0.1361 - mean_squared_error: 0.1361
Epoch 81/150
76/76 [==============================] - 0s 650us/step - loss: 0.1394 - mean_squared_error: 0.1394
Epoch 82/150
76/76 [==============================] - 0s 570us/step - loss: 0.1413 - mean_squared_error: 0.1413
Epoch

76/76 [==============================] - 0s 569us/step - loss: 0.1432 - mean_squared_error: 0.1432
Epoch 147/150
76/76 [==============================] - 0s 609us/step - loss: 0.1372 - mean_squared_error: 0.1372
Epoch 148/150
76/76 [==============================] - 0s 577us/step - loss: 0.1379 - mean_squared_error: 0.1379
Epoch 149/150
76/76 [==============================] - 0s 556us/step - loss: 0.1428 - mean_squared_error: 0.1428
Epoch 150/150
2/2 [==============================] - 0s 2ms/step - loss: 0.1546 - mean_squared_error: 0.1546
test loss, test mse: [0.15458044409751892, 0.15458044409751892]


In [103]:
# run the data on xgboost model
data_dmatrix = xgb.DMatrix(data=X,label=y)

# train xgb classifier
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 100)
xg_reg.fit(X_train,y_train)

preds = xg_reg.predict(X_test)

mse = mean_squared_error(y_test, preds)
print("MSE: %f" % (mse))


MSE: 0.023183


In [104]:
from sklearn import svm
regr = svm.SVR()
regr.fit(X_train, y_train)
preds = regr.predict(X_test)
mse = mean_squared_error(y_test, preds)
print("MSE: %f" % (mse))

MSE: 0.024433


/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [105]:
# Ridge Regression
from sklearn.linear_model import Ridge
clf = Ridge(alpha=1.0)
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
mse = mean_squared_error(y_test, preds)
print("MSE: %f" % (mse))

MSE: 0.024261


In [106]:
# Linear regression to look at coefficient result
import statsmodels.api as sm
#X = sm.add_constant(X.ravel())
results = sm.OLS(y,X).fit()
results.summary()  

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:          ret_0220_0325   R-squared (uncentered):                   0.833
Model:                            OLS   Adj. R-squared (uncentered):              0.830
Method:                 Least Squares   F-statistic:                              309.6
Date:                Mon, 12 Apr 2021   Prob (F-statistic):                        0.00
Time:                        23:42:12   Log-Likelihood:                          423.22
No. Observations:                 946   AIC:                                     -816.4
Df Residuals:                     931   BIC:                                     -743.7
Df Model:                          15                                                  
Covariance Type:            nonrobust                                                  
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
at             -1.265e-07   4.75e-07     -0.266      0.790   -1.06e-06    8.05e-07
lt             -4.937e-07   7.35e-07     -0.671      0.502   -1.94e-06    9.49e-07
che            -6.548e-07   1.18e-06     -0.557      0.578   -2.96e-06    1.65e-06
ni              4.172e-06   3.22e-06      1.295      0.195   -2.15e-06    1.05e-05
sale            7.809e-07   2.75e-07      2.836      0.005    2.41e-07    1.32e-06
execrank          -0.0082      0.006     -1.470      0.142      -0.019       0.003
exchange_id       -0.0789      0.009     -8.554      0.000      -0.097      -0.061
gender_id         -0.0660      0.019     -3.566      0.000      -0.102      -0.030
state_id          -0.0015      0.000     -4.330      0.000      -0.002      -0.001
days_becameceo  8.209e-07   1.72e-06      0.477      0.633   -2.56e-06     4.2e-06
zip            -8.172e-07   1.59e-07     -5.136      0.000   -1.13e-06   -5.05e-07
FF_48             -0.0021      0.000     -5.332      0.000      -0.003      -0.001
shrout_0220     2.757e-08   1.28e-08      2.160      0.031    2.52e-09    5.26e-08
report_length    -3.8e-08   6.34e-08     -0.599      0.549   -1.62e-07    8.65e-08
keyword_perc     -44.6351     15.384     -2.901      0.004     -74.827     -14.443
==============================================================================
Omnibus:                        8.933   Durbin-Watson:                   1.648
Prob(Omnibus):                  0.011   Jarque-Bera (JB):               12.558
Skew:                          -0.056   Prob(JB):                      0.00188
Kurtosis:                       3.553   Cond. No.                     2.02e+09
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[3] The condition number is large, 2.02e+09. This might indicate that there are
strong multicollinearity or other numerical problems.
"""